# Impedance Correction Functions

This code was written for easy EIS data processing in Python. Included below are functions for filtering data via the LinKK test, correcting for inductance (both automatically calculated and known), and removal ohmic resistance (automatically calculated and manual). 

These functions rely on the packages imported below, but particularly the impedance.py package, which is cited here:

Murbach et al., (2020). impedance.py: A Python package for electrochemical impedance analysis. Journal of Open Source Software, 5(52), 2349, https://doi.org/10.21105/joss.02349

In [3]:
import impedance
import csv
import pandas as pd
from impedance.models.circuits import CustomCircuit
import cmath
import numpy as np
import matplotlib.pyplot as plt
from impedance.visualization import plot_nyquist, plot_residuals, plot_bode
from impedance import preprocessing
from impedance.validation import linKK

%matplotlib inline

# Functions

## Manual Inductance Correction
This function takes a known impedance and corrects a given EIS spectrum. Inputs include the frequency array, Z array, and the inductance. The function returns the frequency array and corrected complex impedance array.

In [4]:
def correct_known_inductance(f,Z,L):
    Z_real=np.real(Z)
    Z_imaginary=np.imag(Z)
    Z_imaginary_corrected=[]
    l=len(Z_imaginary)
    for i in range(l):
        Z_imaginary_corrected.append(Z_imaginary[i]-L*2*np.pi*f[i])
        
    Z_corrected=[]
    for i in range(l):
        Z_corrected.append(complex(Z_real[i],Z_imaginary_corrected[i]))
    
    return(np.array(f), np.array(Z_corrected))

## Automatic Inductance Correction
The impedance correction automatically corrects for the impedance of your testing system by fitting f vs. Z_imaginary at high frequency. The range of frequencies can be customized as desired. The function takes a frequency array, Z array (which must be complex), and the minimum and maximum frequencies you'd like to fit at. This function returns the frequency array, corrected impedance (complex), and the inductance L.

Typically, inductance is fit to points where f [Hz] > 10^4.

In [5]:
def correct_inductance(f,Z,f_min, f_max):
    data= {'Frequency': f,
        'Z_real': np.real(Z),
        'Z_imaginary' : np.imag(Z)
        }
    df = pd.DataFrame(data)
    df_correction=df[df['Frequency']>f_min]
    linear_model=np.polyfit(df_correction['Frequency'],df_correction['Z_imaginary'],1)
    p = np.poly1d(linear_model)
    x_s=np.arange(f_min,f_max)
    L=linear_model[0]/(2*np.pi)
    
    Z_imaginary=df['Z_imaginary'].to_numpy()
    
    Z_real=np.real(Z)
    Z_imaginary_corrected=[]
    l=len(Z_imaginary)
    for i in range(l):
        Z_imaginary_corrected.append(Z_imaginary[i]-L*2*np.pi*f[i])
        
    Z_corrected=[]
    for i in range(l):
        Z_corrected.append(complex(Z_real[i],Z_imaginary_corrected[i]))
    
    return(np.array(f), np.array(Z_corrected),L , df['Frequency'], df['Z_imaginary'], x_s, p(x_s))

## Ohmic Resistance Correction
remove_ohmic automatically removes ohmic resistance and any points below the X axis. Input f and Z, returns f_trim and Z_trim. 

***Note: ohmic resistance is automatically calculated to be the Z" value of the highest frequency point that has a positive Z' value, and is approximate. For a more exact correction, fit your spectrum with an equivalent circuit and use remove_ohmic_manual.***

In [6]:
def remove_ohmic(f,Z):
    f_trim, Z_trim = preprocessing.ignoreBelowX(f, Z) #removes Z values where Z[im]<0
    Z_trim_r=np.real(Z_trim)
    Z_trim_i=np.imag(Z_trim)
    R_ohmic=Z_trim_r[0]
    for i in range(len(Z_trim_r)):
        Z_trim_r[i]=Z_trim_r[i]-R_ohmic
        
    for i in range(len(Z_trim_r)):
        Z_trim[i]=complex(Z_trim_r[i],Z_trim_i[i])
    
    return(f_trim,Z_trim)

remove_ohmic_manual removes a given ohmic resistance (R here) from the EIS spectrum. This is handy if you have already fit your EIS spectrum with an equivalent circuit and know R_ohmic already.

In [7]:
#manual ohmic resistance correction
def remove_ohmic_manual(f,Z,R):
    f_trim, Z_trim = preprocessing.ignoreBelowX(f, Z) #removes Z values where Z[im]<0
    Z_trim_r=np.real(Z_trim)
    Z_trim_i=np.imag(Z_trim)
    R_ohmic=R
    for i in range(len(Z_trim_r)):
        Z_trim_r[i]=Z_trim_r[i]-R_ohmic
        
    for i in range(len(Z_trim_r)):
        Z_trim[i]=complex(Z_trim_r[i],Z_trim_i[i])
    
    return(f_trim,Z_trim)

## Outlier correction
The "clean_linkk" function removes any data points that deviate more than a certain percentage from the LinKK fit of our EIS data. The function takes frequency data, Z, and p (which is the maximum fractional acceptable error) and removes points that do not meet this threshold. The function then returns the corrected frequency and Z arrays.

In [8]:
def clean_linkk(f,Z,p):
    M, mu, Z_linKK, res_real, res_imag = linKK(f, Z, c=.5, max_M=100, fit_type='complex', add_cap=True)
    data= {'Frequency': f,
        'Z_real': np.real(Z),
        'Z_imaginary' : np.imag(Z),
        'Real Residuals': res_real,
        'Imaginary Residuals': res_imag
        }
    read = pd.DataFrame(data)
    read_filtered=read[abs(read['Real Residuals']<p)&abs(read['Imaginary Residuals']<p)]
    f_f=read_filtered['Frequency'].to_numpy()
    Z_r_f=read_filtered['Z_real'].to_numpy()
    Z_i_f=read_filtered['Z_imaginary'].to_numpy()
    l_f=len(Z_r_f)
    Z_f=np.zeros(l_f,dtype=complex)

    for i in range(l_f):
        Z_f[i]=complex(Z_r_f[i],Z_i_f[i])
        
    return(np.array(f_f), np.array(Z_f))